<a href="https://colab.research.google.com/github/MarcChen/Fewshot-ICL/blob/main/Text_To_SQL_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Text to SQL


In this lab, we'll apply what we've already learned in the previous labs to build a sqlite database using Ministral 8b. This lab is less guided than the previous ones and you'll need to refer to what you've done previously to complete each part. Moreover, this lab is more focused on prompt engineering and you have to find the best prompt and prompt strategy (system prompt? temperature value? dialog prompt style?).

For this lab, we need to use sqlite3 to execute the generated queries.
Check the doc: https://docs.python.org/3/library/sqlite3.html

<font color='red'>BE CAREFUL: you need to generate sql queries then automaticly exectute them with sqlite3 connector. DO NOT generate python code. DO NOT copy paste genereted query to the connector.</font>

<font color='green'>TIPS: sqlite3 create a file containing your db. Delete it if you need to reset the db.</font>



Lab overview:

0. Modules installation and model loading.
1. Create tables using llm.
2. Populate tables using llm.
3. Explore our tables using llm.
4. More than one table with llm.

IMPORTANT:
- You must work in pairs. You must submit **ONLY ONE NOTEBOOK** for each pair.
- Do not share your work with other pairs.
- You should not use Copilot, ChatGPT or similar tools. At the very least, remove the prompt ...

## 0. Setup

In [1]:
!pip install transformers datasets bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)

from tqdm import tqdm
import pandas as pd
import sqlite3
import torch
import re

In [6]:
# Put your hugging face token here: https://huggingface.co/docs/hub/en/security-tokens
# You need to fill the access form with your huggingface account on this link: https://huggingface.co/mistralai/Ministral-8B-Instruct-2410
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
llm_name = "mistralai/Ministral-8B-Instruct-2410"

# We want to use 4bit quantization to save memory
quantization_config = BitsAndBytesConfig(
    load_in_8bit=False, load_in_4bit=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_name, padding_side="left", token=hf_token)
# Prevent some transformers specific issues.
tokenizer.use_default_system_prompt = False
tokenizer.pad_token_id = tokenizer.eos_token_id

# Load LLM.
llm = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=quantization_config,
    device_map={"": 0}, # load all the model layers on GPU 0
    torch_dtype=torch.bfloat16, # float precision
    token=hf_token
)
# Set LLM on eval mode.
llm.eval()


tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(131072, 4096)
    (layers): ModuleList(
      (0-35): 36 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSN

## 1. Create tables using llms

You need to generate and execute SQL queries to create 3 tables:
- "characters": Id (primary key), Name (str), Age (int), Profession (int).
- "characters20": same than characters.
- "skills": Id (primary key), Name (str), Profession (str).

For example, by running this code ``cursor.execute("""PRAGMA table_info(characters);""").fetchall()``.

You should have this results:

```
`[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 1, None, 0),
 (2, 'age', 'INTEGER', 1, None, 0),
 (3, 'profession', 'TEXT', 1, None, 0)]
 ```

<font color='red'>BE CAREFUL: sqlite3 doesn't have the same possibility than SQL. You may need to specify it.</font>

In [44]:
def generate_sql(prompt, system_prompt="You are an expert SQL developer. Generate only the SQL code without explanations."):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    # Set pad token if not already set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    # Tokenize and prepare inputs
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    # Create attention mask (all ones since we're not padding)
    attention_mask = torch.ones_like(inputs).to("cuda")
    # Generate with attention mask and pad token
    outputs = llm.generate(
        inputs,
        attention_mask=attention_mask,
        max_new_tokens=500,
        temperature=0,
        pad_token_id=tokenizer.pad_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    sql = re.search(r"```sql\n(.*?)\n```", response, re.DOTALL)
    output = sql.group(1).strip() if sql else response
    return output

In [45]:
# Connect to SQLite database
conn = sqlite3.connect('lab_db.sqlite')
cursor = conn.cursor()

# Create tables
tables = [
    ("characters", "id INTEGER PRIMARY KEY, name TEXT NOT NULL, age INTEGER NOT NULL, profession TEXT NOT NULL"),
    ("characters20", "id INTEGER PRIMARY KEY, name TEXT NOT NULL, age INTEGER NOT NULL, profession TEXT NOT NULL"),
    ("skills", "id INTEGER PRIMARY KEY, name TEXT NOT NULL, profession TEXT NOT NULL")
]

for table_name, columns in tables:
    try:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
        prompt = f"Generate a SQLite CREATE TABLE statement for table '{table_name}' with columns: {columns}."
        create_table_sql = generate_sql(prompt)
        cursor.execute(create_table_sql)
        print(f"Table '{table_name}' created successfully.")
    except Exception as e:
        print(f"Error creating table '{table_name}': {e}")

conn.commit()

# Verify table structure
print(cursor.execute("PRAGMA table_info(characters);").fetchall())

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Table 'characters' created successfully.
Table 'characters20' created successfully.
Table 'skills' created successfully.
[(0, 'id', 'INTEGER', 0, None, 1), (1, 'name', 'TEXT', 1, None, 0), (2, 'age', 'INTEGER', 1, None, 0), (3, 'profession', 'TEXT', 1, None, 0)]


## 2. Populate tables using llm

You need to generate and execute SQL queries to fill in “characters” and “characters20” :
- For both, the age must be constrained between 18 and 50 (we'll assess whether the constraint is met later).
- For “characters”, generate 10 rows using the prompt. Apply the prompt 10 times (you should end up with 100 lines).
- For “characters20”, generate 20 rows using the prompt. Apply the prompt 5 times (you should also get 100 lines at the end).


For example, executing this code ``cursor.execute("SELECT * FROM characters")``.

You should get this result (with 100 rows and perhaps different values ...) :

```
[(1, 'Alice', 25, 'Artist'),
 (2, 'Bob', 35, 'Engineer'),
  ...
 (99, 'Ian', 32, 'Architect'),
 (100, 'Jane', 18, 'Dancer')]
 ```

<font color='red'> BE CAREFUL: If your generation configuration doesn't include sampling, you'll always have the same rows.</font>

<font color='green'> BONUS: In section 3, we'll compare the number of duplicated rows between the two methods. Do you have a better strategy for minimizing the number of duplicated rows? Give it a try! (create another table for this purpose) </font>

In [46]:
def insert_data(table_name, num_rows_per_batch, num_batches):
    for _ in tqdm(range(num_batches)):
        prompt = f"""
        Generate a SQL INSERT statement to add {num_rows_per_batch} rows into the '{table_name}' table.
        Columns: name (random realistic names), age (18-50), profession (common jobs).
        Example: INSERT INTO {table_name} (name, age, profession) VALUES ('Alice', 25, 'Engineer'), ...;
        """
        system_prompt = "You are a SQL expert. Generate only the INSERT statement with valid SQL syntax."
        try:
            insert_sql = generate_sql(prompt, system_prompt)
            cursor.executescript(insert_sql)  # Use executescript to handle multiple rows
            conn.commit()
        except Exception as e:
            print(f"Error inserting data: {e}")

# Insert data for 'characters' (10x10)
insert_data("characters", num_rows_per_batch=10, num_batches=10)

# Insert data for 'characters20' (20x5)
insert_data("characters20", num_rows_per_batch=20, num_batches=5)

# Bonus: Alternative strategy to minimize duplicates (e.g., UUID-based)
cursor.execute("CREATE TABLE IF NOT EXISTS characters_bonus (id INTEGER PRIMARY KEY, name TEXT, age INTEGER, profession TEXT);")

 20%|██        | 2/10 [00:32<02:11, 16.46s/it]


KeyboardInterrupt: 

In [ ]:
# Verify table structure
print(cursor.execute("SELECT * FROM characters;").fetchall())

## 3. Explore our tables using llm.
  
First, you need to generate and execute SQL queries that indicate the number of duplicate rows (without ids) in each character table. To make things easier, we only ask for the number of each duplicated rows.

Here is an examples of expected results:

```
[(2,), (7,), (5,), (2,), (2,), (3,), (2,), (2,), (2,), (2,), (2,), (2,), (2,)]
```

<font color='green'> BONUS: Generate a query that returns the total count of duplicated rows. You may need to do this in several steps.</font>

Secondly, you need to generate and execute SQL queries that remove duplicate rows. To make things easier, it's not necessary to keep original duplicated lines. For example, if you have a list like this : [a, b, a, c]. We ask you to remove all the a: [b, c].

<font color='green'> BONUS: Generate a query that delete duplicated but keep the original row. [a, b, a, c] -> [a, b, c] </font>

Finaly, you need to generate and execute SQL queries that check if the age constraint is respected.

<font color='red'> BE CAREFUL: Do each step for every characters tables you have.</font>


In [34]:
def count_duplicates(table_name):
    query = f"""
    SELECT COUNT(*) AS duplicate_count
    FROM {table_name}
    GROUP BY name, age, profession
    HAVING COUNT(*) > 1;
    """
    cursor.execute(query)
    return cursor.fetchall()

print("Duplicates in 'characters':", count_duplicates("characters"))
print("Duplicates in 'characters20':", count_duplicates("characters20"))

Duplicates in 'characters': []
Duplicates in 'characters20': []


In [33]:
def remove_duplicates(table_name):
    query = f"""
    DELETE FROM {table_name}
    WHERE rowid NOT IN (
        SELECT MIN(rowid)
        FROM {table_name}
        GROUP BY name, age, profession
    );
    """
    cursor.execute(query)
    conn.commit()

remove_duplicates("characters")
remove_duplicates("characters20")

In [35]:
def check_age_constraints(table_name):
    cursor.execute(f"SELECT COUNT(*) FROM {table_name} WHERE age < 18 OR age > 50;")
    return cursor.fetchone()[0]

print("Invalid ages in 'characters':", check_age_constraints("characters"))

Invalid ages in 'characters': 0


## 4. More than one table with llm.
  
First, choose your best characters table (with the largest number of rows).


Second, generate and execute an SQL query that returns the set of unique professions in the table.


Third, generate and execute an SQL query that populates the skill tables from this set of unique professions.


Fourth, generate and execute an SQL query that verifies that the professions in the skill table exist in your characters table.


Finally, generate and execute an SQL query that returns the name of the skills associated with a character name (by profession).

# 1. Choosing best table

In [43]:
def count_rows(table_name):
    query = f"""
    SELECT COUNT(*)
    FROM {table_name}
    """
    cursor.execute(query)
    return cursor.fetchall()

character_rows = count_rows("characters")[0][0]
character20_rows = count_rows("characters20")[0][0]
best_table = "characters20" if character20_rows > character_rows else "characters"
print(f"Best table is: {best_table}")

ProgrammingError: Cannot operate on a closed database.

In [42]:
# Get unique professions from the best table (e.g., characters20)
cursor.execute("SELECT DISTINCT profession FROM characters20;")
professions = [row[0] for row in cursor.fetchall()]

# Populate skills table
for profession in professions:
    cursor.execute(f"INSERT INTO skills (name, profession) VALUES ('Skill for {profession}', ?)", (profession,))
conn.commit()

# Verify professions exist in both tables
cursor.execute("""
SELECT DISTINCT s.profession
FROM skills s
WHERE EXISTS (
    SELECT 1 FROM characters20 c WHERE c.profession = s.profession
);
""")
print("Valid professions:", cursor.fetchall())

# Get skills for a character (e.g., 'Alice')
cursor.execute("""
SELECT c.name, s.name
FROM characters20 c
JOIN skills s ON c.profession = s.profession
WHERE c.name = 'Alice';
""")
print("Skills for Alice:", cursor.fetchall())

conn.close()

Valid professions: []
Skills for Alice: []
